# 네이버 스포츠 월간 승패 기록 크롤링

## 2016년 SK의 정규시즌 월간 기록 크롤링
### 1. 필요한 패키지 불러오기

In [1]:
import math
import re
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from bs4 import BeautifulSoup as bs
from sklearn import svm
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import learning_curve

import html5lib

from sklearn.svm import SVC

### 2. 링크에서 컨텐츠 불러오기
<strong>Response 200으로 제대로 불러왔는지 확인

In [2]:
r = requests.get("https://sports.news.naver.com/kbaseball/schedule/index.nhn?date=20210320&month=04&year=2016&teamCode=SK")
r

<Response [200]>

In [3]:
root = bs(r.content, "html.parser")
table = root.find("div", {"class":"tb_wrap"})
table

<div class="tb_wrap" id="calendarWrap">
<div class="sch_tb">
<table border="1" cellspacing="0">
<colgroup>
<col width="107"/><col width="55"/><col width="300"/><col width="200"/><col width="162"/><col width="74"/><col width=""/>
</colgroup>
<tbody>
<tr>
<td rowspan="1">
<span class="td_date"><strong>4.1</strong> (금)</span>
</td>
<td><span class="td_hour">19:00</span></td>
<td class=""><span class="team_lft">kt</span>
<img alt="kt" height="25" src="https://dthumb-phinf.pstatic.net/?src=https://sports-phinf.pstatic.net/team/kbo/default/KT.png&amp;type=f25_25&amp;refresh=1" title="kt" width="25"/>
<strong class="td_score">8<em>:</em>4</strong>
<img alt="SK" height="25" src="https://dthumb-phinf.pstatic.net/?src=https://sports-phinf.pstatic.net/team/kbo/default/SK.png&amp;type=f25_25&amp;refresh=1" title="SK" width="25"/>
<span class="team_rgt">SK</span>
</td>
<td>
<span class="td_btn">
<!-- 문자중계 경기결과 버튼 -->
<a href="/gameCenter/gameResult.nhn?category=kbo&amp;gameId=20160401KTSK02016" onc

### 3. 불러온 컨텐츠에서 필요한 부분 태그 확인,  필터링하기

<strong>경기 날짜와 결과 부분 리스트에 담기

In [4]:
lines = table.find_all('tr')
lines

date = []
score = []

for line in lines:
    td_list = line.find_all('td')
    #print(td_list)
    d = str(td_list[0].text)
    dt = d.strip('\n')
    date.append(dt)
    s = str(td_list[2].text)
    scr = s.replace('\n',' ')
    score.append(scr)
    
score        


['kt  8:4  SK ',
 'kt  3:4  SK ',
 'kt  5:4  SK ',
 '프로야구 경기가 없습니다.',
 'SK  1:2  롯데 ',
 'SK  1:11  롯데 ',
 'SK  8:3  롯데 ',
 'LG  2:3  SK ',
 'LG  3:4  SK ',
 'LG  6:7  SK ',
 '프로야구 경기가 없습니다.',
 'KIA  7:6  SK ',
 'KIA  0:2  SK ',
 'KIA  6:7  SK ',
 'SK  7:3  kt ',
 'SK  VS  kt 경기취소 ',
 'SK  10:6  kt ',
 '프로야구 경기가 없습니다.',
 '넥센  3:1  SK ',
 '넥센  1:9  SK ',
 '넥센  2:3  SK ',
 'NC  6:5  SK ',
 'NC  2:8  SK ',
 'NC  2:3  SK ',
 '프로야구 경기가 없습니다.',
 'SK  3:4  두산 ',
 'SK  3:1  두산 ',
 'SK  1:4  두산 ',
 'SK  4:3  넥센 ',
 'SK  5:1  넥센 ']

<strong>+)최종적으로 데이터프레임에 같이 담을 것이므로 리스트 갯수가 같은지 확인하기

In [5]:
len(date)
len(score)
type(score)

list

### 4. 첫번째 데이터 프레임 생성하기

In [6]:
raw = pd.DataFrame({'Date':date, 'Score':score})
raw


,Date,Score
0,4.1 (금),kt 8:4 SK
1,4.2 (토),kt 3:4 SK
2,4.3 (일),kt 5:4 SK
3,4.4 (월),프로야구 경기가 없습니다.
4,4.5 (화),SK 1:2 롯데
5,4.6 (수),SK 1:11 롯데
6,4.7 (목),SK 8:3 롯데
7,4.8 (금),LG 2:3 SK
8,4.9 (토),LG 3:4 SK
9,4.10 (일),LG 6:7 SK


### 5. 하나의 문자열로 되어있는 스코어 기록을 각 팀/ 각 팀의 경기 스코어/승자/패자 열로 새로 생성하기

양 팀으로 나누기

In [7]:
team_L = []
team_R = []

for i in raw['Score']:
    if "경기" not in i:
        team_L.append(i.split(":")[0])
        team_R.append(i.split(":")[1])
    
    else:
        team_L.append('-')
        team_R.append('-')


<strong>+)타입이 pandas.core.series.Series 일때는 반복문으로 원소 하나씩 가져오기 가능, list 일때는 반복문에서 문자 원소 하나씩 가져올 수 없음 
 
    
즉, raw[새로운 컬럼] = 리스트 형식으로 추가하지 말고 pandas 함수 사용하여 추가해야 함.

In [8]:
raw = raw.assign(Team_L = team_L, Team_R = team_R)
raw


,Date,Score,Team_L,Team_R
0,4.1 (금),kt 8:4 SK,kt 8,4 SK
1,4.2 (토),kt 3:4 SK,kt 3,4 SK
2,4.3 (일),kt 5:4 SK,kt 5,4 SK
3,4.4 (월),프로야구 경기가 없습니다.,-,-
4,4.5 (화),SK 1:2 롯데,SK 1,2 롯데
5,4.6 (수),SK 1:11 롯데,SK 1,11 롯데
6,4.7 (목),SK 8:3 롯데,SK 8,3 롯데
7,4.8 (금),LG 2:3 SK,LG 2,3 SK
8,4.9 (토),LG 3:4 SK,LG 3,4 SK
9,4.10 (일),LG 6:7 SK,LG 6,7 SK


팀명과 스코어 나누기

In [9]:
#다시 빈칸을 기준으로 팀과 스코어 나눠줌

score_L = []
score_R = []
team_left = []
team_right = []

for i in raw['Team_L']:
    if "-" not in i:
        score_L.append(i.split()[1])
        team_left.append(i.split()[0])
    
    else:
        score_L.append(i)
        team_left.append(i)

for i in raw['Team_R']:
    if "-" not in i:
        score_R.append(i.split()[0])
        team_right.append(i.split()[1])
    else:
        score_R.append(i)
        team_right.append(i)


In [10]:
raw = raw.assign(Score_L = score_L, Score_R = score_R, Team_Left = team_left, Team_Right = team_right)
raw


,Date,Score,Team_L,Team_R,Score_L,Score_R,Team_Left,Team_Right
0,4.1 (금),kt 8:4 SK,kt 8,4 SK,8,4,kt,SK
1,4.2 (토),kt 3:4 SK,kt 3,4 SK,3,4,kt,SK
2,4.3 (일),kt 5:4 SK,kt 5,4 SK,5,4,kt,SK
3,4.4 (월),프로야구 경기가 없습니다.,-,-,-,-,-,-
4,4.5 (화),SK 1:2 롯데,SK 1,2 롯데,1,2,SK,롯데
5,4.6 (수),SK 1:11 롯데,SK 1,11 롯데,1,11,SK,롯데
6,4.7 (목),SK 8:3 롯데,SK 8,3 롯데,8,3,SK,롯데
7,4.8 (금),LG 2:3 SK,LG 2,3 SK,2,3,LG,SK
8,4.9 (토),LG 3:4 SK,LG 3,4 SK,3,4,LG,SK
9,4.10 (일),LG 6:7 SK,LG 6,7 SK,6,7,LG,SK


승리팀, 패배팀 나누기

In [11]:
#마지막으로 승패 결과 추가

win = []
lose = []

for i,j,a,b in zip(raw['Score_L'],raw['Score_R'],raw['Team_Left'],raw['Team_Right']):
    if i > j:
        win.append(a)
        lose.append(b)
    elif j > i:
        win.append(b)
        lose.append(a)
    else:
        win.append('-')
        lose.append('-')
        
raw = raw.assign(Win = win, Lose = lose)
raw


,Date,Score,Team_L,Team_R,Score_L,Score_R,Team_Left,Team_Right,Win,Lose
0,4.1 (금),kt 8:4 SK,kt 8,4 SK,8,4,kt,SK,kt,SK
1,4.2 (토),kt 3:4 SK,kt 3,4 SK,3,4,kt,SK,SK,kt
2,4.3 (일),kt 5:4 SK,kt 5,4 SK,5,4,kt,SK,kt,SK
3,4.4 (월),프로야구 경기가 없습니다.,-,-,-,-,-,-,-,-
4,4.5 (화),SK 1:2 롯데,SK 1,2 롯데,1,2,SK,롯데,롯데,SK
5,4.6 (수),SK 1:11 롯데,SK 1,11 롯데,1,11,SK,롯데,롯데,SK
6,4.7 (목),SK 8:3 롯데,SK 8,3 롯데,8,3,SK,롯데,SK,롯데
7,4.8 (금),LG 2:3 SK,LG 2,3 SK,2,3,LG,SK,SK,LG
8,4.9 (토),LG 3:4 SK,LG 3,4 SK,3,4,LG,SK,SK,LG
9,4.10 (일),LG 6:7 SK,LG 6,7 SK,6,7,LG,SK,SK,LG


### 6. 저장명에 년도, 월, 팀명 넣어서 저장하기

In [119]:
#파일로 저장
raw.to_excel('2016_04_SK.xlsx')


----------------------------------------------
## +) 여기까지의 과정을 하나의 반복문에 담아서 한 해의 결과를 한번의 과정으로 나타낸다.

In [ ]:

#2016 SK 정규시즌 월별 경기 기록 불러오기. 파일 저장까지
#2016 정규시즌은 04-10월

for i in ['04','05','06','07','08','09','10']:
    link = "https://sports.news.naver.com/kbaseball/schedule/index.nhn?date=20210320"
    params = {'month':i, 'year':'2016', 'teamCode':'SK'}
    headers = {'user-agent': '헤더정보'}
    
    r = requests.get(link, params=params, headers=headers)
    root = bs(r.content, "html.parser")
    table = root.find("div", {"class":"tb_wrap"})
    lines = table.find_all('tr')
    
    
    date = []
    score = []

    for line in lines:
        td_list = line.find_all('td')
        d = str(td_list[0].text)
        dt = d.strip('\n')
        date.append(dt)
        s = str(td_list[2].text)
        scr = s.replace('\n',' ')
        score.append(scr)
        
    raw = pd.DataFrame({'Date':date, 'Score':score})
    
    team_L = []
    team_R = []

    for a in raw['Score']:
        if "경기" not in a:
            team_L.append(a.split(":")[0])
            team_R.append(a.split(":")[1])

        else:
            team_L.append('-')
            team_R.append('-')
            
    raw = raw.assign(Team_L = team_L, Team_R = team_R)
    
    score_L = []
    score_R = []
    team_left = []
    team_right = []

    for b in raw['Team_L']:
        if "-" not in b:
            score_L.append(b.split()[1])
            team_left.append(b.split()[0])

        else:
            score_L.append(b)
            team_left.append(b)

    for c in raw['Team_R']:
        if "-" not in c:
            score_R.append(c.split()[0])
            team_right.append(c.split()[1])
        else:
            score_R.append(c)
            team_right.append(c)
            
    raw = raw.assign(Score_L = score_L, Score_R = score_R, Team_Left = team_left, Team_Right = team_right)
    
    win = []
    lose = []

    for d,e,f,g in zip(raw['Score_L'],raw['Score_R'],raw['Team_Left'],raw['Team_Right']):
        if d > e:
            win.append(f)
            lose.append(g)
        elif e > d:
            win.append(g)
            lose.append(f)
        else:
            win.append('-')
            lose.append('-')

    raw = raw.assign(Win = win, Lose = lose)
    
    raw.to_excel('2016_{}_SK.xlsx'.format(i))


